# Datawhale 零基础入门数据挖掘-Baseline

## Baseline-v1.0 版

Tip:这是一个最初始baseline版本,抛砖引玉,为大家提供一个基本Baseline和一个竞赛流程的基本介绍，欢迎大家多多交流。

**赛题：零基础入门数据挖掘 - 二手车交易价格预测**

地址：https://tianchi.aliyun.com/competition/entrance/231784/introduction?spm=5176.12281957.1004.1.38b02448ausjSX

### Step 1:导入函数工具箱

In [1]:
## 基础工具
import numpy as np
import pandas as pd
import warnings
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import jn
from IPython.display import display, clear_output
import time

warnings.filterwarnings('ignore')
%matplotlib inline

## 模型预测的
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
 
## 数据降维处理的
from sklearn.decomposition import PCA,FastICA,FactorAnalysis,SparsePCA

import lightgbm as lgb
import xgboost as xgb

## 参数搜索和评价的
from sklearn.model_selection import GridSearchCV,cross_val_score,StratifiedKFold,train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

### Step 2:数据读取

In [2]:
## 通过Pandas对于数据进行读取 (pandas是一个很友好的数据读取函数库)
# Train_data = pd.read_csv('datalab/231784/used_car_train_20200313.csv', sep=' ')
# TestA_data = pd.read_csv('datalab/231784/used_car_testA_20200313.csv', sep=' ')
Train_data = pd.read_csv('dataset/used_car_train_20200313.csv', sep=' ')
TestA_data = pd.read_csv('dataset//used_car_testA_20200313.csv', sep=' ')
## 输出数据的大小信息
print('Train data shape:',Train_data.shape)
print('TestA data shape:',TestA_data.shape)

Train data shape: (150000, 31)
TestA data shape: (50000, 30)


#### 1) 数据简要浏览

In [3]:
## 通过.head() 简要浏览读取数据的形式
Train_data.head()

,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,...,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
0,0,736,20040402,30.0,6,1.0,0.0,0.0,60,12.5,...,0.235676,0.101988,0.129549,0.022816,0.097462,-2.881803,2.804097,-2.420821,0.795292,0.914762
1,1,2262,20030301,40.0,1,2.0,0.0,0.0,0,15.0,...,0.264777,0.121004,0.135731,0.026597,0.020582,-4.900482,2.096338,-1.030483,-1.722674,0.245522
2,2,14874,20040403,115.0,15,1.0,0.0,0.0,163,12.5,...,0.251410,0.114912,0.165147,0.062173,0.027075,-4.846749,1.803559,1.565330,-0.832687,-0.229963
3,3,71865,19960908,109.0,10,0.0,0.0,1.0,193,15.0,...,0.274293,0.110300,0.121964,0.033395,0.000000,-4.509599,1.285940,-0.501868,-2.438353,-0.478699
4,4,111080,20120103,110.0,5,1.0,0.0,0.0,68,5.0,...,0.228036,0.073205,0.091880,0.078819,0.121534,-1.896240,0.910783,0.931110,2.834518,1.923482


#### 2) 数据信息查看

In [4]:
## 通过 .info() 简要可以看到对应一些数据列名，以及NAN缺失信息
Train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 31 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   SaleID             150000 non-null  int64  
 1   name               150000 non-null  int64  
 2   regDate            150000 non-null  int64  
 3   model              149999 non-null  float64
 4   brand              150000 non-null  int64  
 5   bodyType           145494 non-null  float64
 6   fuelType           141320 non-null  float64
 7   gearbox            144019 non-null  float64
 8   power              150000 non-null  int64  
 9   kilometer          150000 non-null  float64
 10  notRepairedDamage  150000 non-null  object 
 11  regionCode         150000 non-null  int64  
 12  seller             150000 non-null  int64  
 13  offerType          150000 non-null  int64  
 14  creatDate          150000 non-null  int64  
 15  price              150000 non-null  int64  
 16  v_

In [5]:
## 通过 .columns 查看列名
Train_data.columns

Index(['SaleID', 'name', 'regDate', 'model', 'brand', 'bodyType', 'fuelType',
       'gearbox', 'power', 'kilometer', 'notRepairedDamage', 'regionCode',
       'seller', 'offerType', 'creatDate', 'price', 'v_0', 'v_1', 'v_2', 'v_3',
       'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12',
       'v_13', 'v_14'],
      dtype='object')

In [6]:
TestA_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SaleID             50000 non-null  int64  
 1   name               50000 non-null  int64  
 2   regDate            50000 non-null  int64  
 3   model              50000 non-null  float64
 4   brand              50000 non-null  int64  
 5   bodyType           48587 non-null  float64
 6   fuelType           47107 non-null  float64
 7   gearbox            48090 non-null  float64
 8   power              50000 non-null  int64  
 9   kilometer          50000 non-null  float64
 10  notRepairedDamage  50000 non-null  object 
 11  regionCode         50000 non-null  int64  
 12  seller             50000 non-null  int64  
 13  offerType          50000 non-null  int64  
 14  creatDate          50000 non-null  int64  
 15  v_0                50000 non-null  float64
 16  v_1                500

#### 3) 数据统计信息浏览

In [7]:
## 通过 .describe() 可以查看数值特征列的一些统计信息
Train_data.describe()

,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,...,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
count,150000.000000,150000.000000,1.500000e+05,149999.000000,150000.000000,145494.000000,141320.000000,144019.000000,150000.000000,150000.000000,...,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000
mean,74999.500000,68349.172873,2.003417e+07,47.129021,8.052733,1.792369,0.375842,0.224943,119.316547,12.597160,...,0.248204,0.044923,0.124692,0.058144,0.061996,-0.001000,0.009035,0.004813,0.000313,-0.000688
std,43301.414527,61103.875095,5.364988e+04,49.536040,7.864956,1.760640,0.548677,0.417546,177.168419,3.919576,...,0.045804,0.051743,0.201410,0.029186,0.035692,3.772386,3.286071,2.517478,1.288988,1.038685
min,0.000000,0.000000,1.991000e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,-9.168192,-5.558207,-9.639552,-4.153899,-6.546556
25%,37499.750000,11156.000000,1.999091e+07,10.000000,1.000000,0.000000,0.000000,0.000000,75.000000,12.500000,...,0.243615,0.000038,0.062474,0.035334,0.033930,-3.722303,-1.951543,-1.871846,-1.057789,-0.437034
50%,74999.500000,51638.000000,2.003091e+07,30.000000,6.000000,1.000000,0.000000,0.000000,110.000000,15.000000,...,0.257798,0.000812,0.095866,0.057014,0.058484,1.624076,-0.358053,-0.130753,-0.036245,0.141246
75%,112499.250000,118841.250000,2.007111e+07,66.000000,13.000000,3.000000,1.000000,0.000000,150.000000,15.000000,...,0.265297,0.102009,0.125243,0.079382,0.087491,2.844357,1.255022,1.776933,0.942813,0.680378
max,149999.000000,196812.000000,2.015121e+07,247.000000,39.000000,7.000000,6.000000,1.000000,19312.000000,15.000000,...,0.291838,0.151420,1.404936,0.160791,0.222787,12.357011,18.819042,13.847792,11.147669,8.658418


In [8]:
TestA_data.describe()

,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,...,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
count,50000.000000,50000.000000,5.000000e+04,50000.000000,50000.000000,48587.000000,47107.000000,48090.000000,50000.000000,50000.000000,...,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,174999.500000,68542.223280,2.003393e+07,46.844520,8.056240,1.782185,0.373405,0.224350,119.883620,12.595580,...,0.248669,0.045021,0.122744,0.057997,0.062000,-0.017855,-0.013742,-0.013554,-0.003147,0.001516
std,14433.901067,61052.808133,5.368870e+04,49.469548,7.819477,1.760736,0.546442,0.417158,185.097387,3.908979,...,0.044601,0.051766,0.195972,0.029211,0.035653,3.747985,3.231258,2.515962,1.286597,1.027360
min,150000.000000,0.000000,1.991000e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,-9.160049,-5.411964,-8.916949,-4.123333,-6.112667
25%,162499.750000,11203.500000,1.999091e+07,10.000000,1.000000,0.000000,0.000000,0.000000,75.000000,12.500000,...,0.243762,0.000044,0.062644,0.035084,0.033714,-3.700121,-1.971325,-1.876703,-1.060428,-0.437920
50%,174999.500000,52248.500000,2.003091e+07,29.000000,6.000000,1.000000,0.000000,0.000000,109.000000,15.000000,...,0.257877,0.000815,0.095828,0.057084,0.058764,1.613212,-0.355843,-0.142779,-0.035956,0.138799
75%,187499.250000,118856.500000,2.007110e+07,65.000000,13.000000,3.000000,1.000000,0.000000,150.000000,15.000000,...,0.265328,0.102025,0.125438,0.079077,0.087489,2.832708,1.262914,1.764335,0.941469,0.681163
max,199999.000000,196805.000000,2.015121e+07,246.000000,39.000000,7.000000,6.000000,1.000000,20000.000000,15.000000,...,0.291618,0.153265,1.358813,0.156355,0.214775,12.338872,18.856218,12.950498,5.913273,2.624622


### Step 3:特征与标签构建

#### 1) 提取数值类型特征列名

In [9]:
numerical =  ['power', 'kilometer', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13','v_14' ]
categorical= ['name', 'model', 'brand', 'bodyType', 'fuelType', 'gearbox', 'notRepairedDamage', 'regionCode',]

In [10]:
Train_data["notRepairedDamage"].replace("-",np.nan,inplace=True)
TestA_data["notRepairedDamage"].replace("-",np.nan,inplace=True)

In [11]:
Train_data = Train_data.drop(["offerType","seller"],axis=1)
TestA_data = TestA_data.drop(["offerType","seller"],axis=1)

In [12]:
upper_Tr = Train_data.power.quantile(0.75)+(Train_data.power.quantile(0.75)-Train_data.power.quantile(0.25))*1.5
upper_Te = TestA_data.power.quantile(0.75)+(TestA_data.power.quantile(0.75)-TestA_data.power.quantile(0.25))*1.5
upper_Te

262.5

In [67]:
def outliers_proc(data, col_name, scale=3):
    """
        用于截尾异常值， 默认用box_plot(scale=3)进行清洗
        param:
            data：接收pandas数据格式
            col_name: pandas列名
            scale: 尺度
    """
    data_col = data[col_name]
    Q1 = data_col.quantile(0.25) # 0.25分位数
    Q3 = data_col.quantile(0.75)  # 0,75分位数
    IQR = Q3 - Q1

    data_col[data_col < Q1 - (scale * IQR)] = Q1 - (scale * IQR)
    data_col[data_col > Q3 + (scale * IQR)] = Q3 + (scale * IQR)

    return data[col_name]


Train_data['power']=outliers_proc(Train_data,'power')
TestA_data['power']=outliers_proc(TestA_data,'power')

In [68]:
Train_data.columns

Index(['SaleID', 'name', 'regDate', 'model', 'brand', 'bodyType', 'fuelType',
       'gearbox', 'power', 'kilometer', 'notRepairedDamage', 'regionCode',
       'creatDate', 'price', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6',
       'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14'],
      dtype='object')

In [69]:
y = Train_data["price"]
train = Train_data.drop("price",axis=1)
train["train"]=1
TestA_data["train"] = 0

In [70]:
print(train.shape);print(TestA_data.shape)

(150000, 29)
(50000, 29)


In [71]:
data = pd.concat([train,TestA_data],ignore_index=True)
data.columns

Index(['SaleID', 'name', 'regDate', 'model', 'brand', 'bodyType', 'fuelType',
       'gearbox', 'power', 'kilometer', 'notRepairedDamage', 'regionCode',
       'creatDate', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6', 'v_7',
       'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14', 'train'],
      dtype='object')

In [72]:
data['used_time'] = (pd.to_datetime(data['creatDate'], format='%Y%m%d', errors='coerce') - 
                            pd.to_datetime(data['regDate'], format='%Y%m%d', errors='coerce')).dt.days

In [73]:
data['used_time'].isnull().sum()

15101

In [74]:
Train_gb = Train_data.groupby("brand")
all_info = {}
for kind, kind_data in Train_gb:
    info = {}
    kind_data = kind_data[kind_data['price'] > 0]
    info['brand_amount'] = len(kind_data)
    info['brand_price_max'] = kind_data.price.max()
    info['brand_price_median'] = kind_data.price.median()
    info['brand_price_min'] = kind_data.price.min()
    info['brand_price_sum'] = kind_data.price.sum()
    info['brand_price_std'] = kind_data.price.std()
    info['brand_price_average'] = round(kind_data.price.sum() / (len(kind_data) + 1), 2)
    all_info[kind] = info
brand_fe = pd.DataFrame(all_info).T.reset_index().rename(columns={"index": "brand"})
data = data.merge(brand_fe, how='left', on='brand')

In [75]:
bin = [i*10 for i in range(31)]
data['power_bin'] = pd.cut(data['power'], bin, labels=False)
data[['power_bin', 'power']].head()

,power_bin,power
0,5.0,60.0
1,NaN,0.0
2,16.0,163.0
3,19.0,193.0
4,6.0,68.0


In [76]:
data = data.drop(['creatDate', 'regDate', 'regionCode'], axis=1)

In [77]:
print(data.shape)
data.columns

(200000, 35)


Index(['SaleID', 'name', 'model', 'brand', 'bodyType', 'fuelType', 'gearbox',
       'power', 'kilometer', 'notRepairedDamage', 'v_0', 'v_1', 'v_2', 'v_3',
       'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12',
       'v_13', 'v_14', 'train', 'used_time', 'brand_amount', 'brand_price_max',
       'brand_price_median', 'brand_price_min', 'brand_price_sum',
       'brand_price_std', 'brand_price_average', 'power_bin'],
      dtype='object')

In [78]:
data.to_csv('data_for_tree.csv', index=0)

In [79]:
train_data = data[data["train"]==1]
test_data = data[data["train"]==0]

In [80]:
train_data = train_data.drop(['train','SaleID','name'],axis=1)
test_data = test_data.drop(['train','SaleID','name'],axis=1)

In [81]:
X_data = train_data.fillna(-1)
X_test = test_data.fillna(-1)

In [82]:
# from sklearn.

In [83]:
y = np.log(y+1)

In [84]:
X_data = pd.get_dummies(X_data)
X_test = pd.get_dummies(X_test)

In [85]:
X_data

,model,brand,bodyType,fuelType,gearbox,power,kilometer,v_0,v_1,v_2,...,brand_price_max,brand_price_median,brand_price_min,brand_price_sum,brand_price_std,brand_price_average,power_bin,notRepairedDamage_-1,notRepairedDamage_0.0,notRepairedDamage_1.0
0,30.0,6,1.0,0.0,0.0,60.0,12.5,43.357796,3.966344,0.050257,...,59900.0,1800.0,13.0,36902172.0,4681.293524,3611.49,5.0,0,1,0
1,40.0,1,2.0,0.0,0.0,0.0,15.0,45.305273,5.236112,0.137925,...,99900.0,6499.0,15.0,127916065.0,9369.631497,9272.64,-1.0,1,0,0
2,115.0,15,1.0,0.0,0.0,163.0,12.5,45.978359,4.823792,1.319524,...,45000.0,8500.0,100.0,14373814.0,5425.058140,9851.83,16.0,0,1,0
3,109.0,10,0.0,0.0,1.0,193.0,15.0,45.687478,4.492574,-0.050616,...,98000.0,5400.0,15.0,120700489.0,8988.307535,8470.21,19.0,0,1,0
4,110.0,5,1.0,0.0,0.0,68.0,5.0,44.383511,2.031433,0.572169,...,31500.0,2300.0,20.0,15424120.0,3343.624586,3305.64,6.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,121.0,10,4.0,0.0,1.0,163.0,15.0,45.316543,-3.139095,-1.269707,...,98000.0,5400.0,15.0,120700489.0,8988.307535,8470.21,16.0,0,1,0
149996,116.0,11,0.0,0.0,0.0,125.0,10.0,45.972058,-3.143764,-0.023523,...,34500.0,2900.0,30.0,13398956.0,4721.824690,4548.19,12.0,0,1,0
149997,60.0,11,1.0,1.0,0.0,90.0,6.0,44.733481,-3.105721,0.595454,...,34500.0,2900.0,30.0,13398956.0,4721.824690,4548.19,8.0,0,1,0
149998,34.0,10,3.0,1.0,0.0,156.0,15.0,45.658634,-3.204785,-0.441680,...,98000.0,5400.0,15.0,120700489.0,8988.307535,8470.21,15.0,0,1,0


In [86]:
train_x = X_data[:int(len(X_data)*0.8)]
valid_x = X_data[int(len(X_data)*0.8):]
train_y = y[:int(len(X_data)*0.8)]
valid_y = y[int(len(X_data)*0.8):]

In [87]:
x_train,x_test,y_train,y_test = train_test_split(train_x,train_y,test_size = 0.2)

In [88]:
print(x_train.shape)
print(y_train.shape)

(96000, 34)
(96000,)


In [89]:
X_test.shape

(50000, 34)

In [90]:
TestA_data.shape

(50000, 29)

In [91]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error,  make_scorer

In [92]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

In [93]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from xgboost.sklearn import XGBRegressor
from lightgbm.sklearn import LGBMRegressor

In [94]:
def getNat(nums):
    return np.exp(nums)+1

max_depth=14  
lr =0.09  
subsample=1  
colsample_bylevel=0.4  
reg_lambda=0.4  
550.49  

In [95]:
i= 0.4
xgb = XGBRegressor(n_estimators = 100, objective='reg:squarederror',learning_rate=0.09,max_depth=14,gamma=0,subsample=1,colsample_bylevel=0.4,reg_lambda=0.4,random_seed=1 )
xgb.fit(x_train,y_train)
print("{0} is {1}".format(i,mean_absolute_error(getNat(xgb.predict(valid_x)),getNat(valid_y))))
print("test set : {0} is {1}".format(i,mean_absolute_error(getNat(xgb.predict(x_test)),getNat(y_test))))

0.4 is 558.6831814166069
test set : 0.4 is 562.795652435422


In [42]:
# for i in np.arange(0.01,0.1,0.01):
#     xgb = XGBRegressor(n_estimators = 100, objective='reg:squarederror',learning_rate=i,max_depth=14,gamma=0,subsample=1,colsample_bylevel=0.4,reg_lambda=i  )
#     xgb.fit(x_train,y_train)
#     print("{0} is {1}".format(i,mean_absolute_error(getNat(xgb.predict(valid_x)),getNat(valid_y))))
#     print("test set : {0} is {1}".format(i,mean_absolute_error(getNat(xgb.predict(x_test)),getNat(y_test))))

In [96]:
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)

In [97]:
gbm = LGBMRegressor(objective='regression', num_leaves=20, learning_rate=0.1, n_estimators=100,bagging_fraction=0.8)
gbm.fit(x_train, y_train, eval_set=[(x_test, y_test)], eval_metric="mean_absolute_error")

[1]	valid_0's l1: 0.902249	valid_0's l2: 1.2295
[2]	valid_0's l1: 0.821544	valid_0's l2: 1.02381
[3]	valid_0's l1: 0.749445	valid_0's l2: 0.856018
[4]	valid_0's l1: 0.685097	valid_0's l2: 0.719037
[5]	valid_0's l1: 0.627454	valid_0's l2: 0.607082
[6]	valid_0's l1: 0.576613	valid_0's l2: 0.516199
[7]	valid_0's l1: 0.53126	valid_0's l2: 0.441639
[8]	valid_0's l1: 0.490889	valid_0's l2: 0.380587
[9]	valid_0's l1: 0.454598	valid_0's l2: 0.330001
[10]	valid_0's l1: 0.422437	valid_0's l2: 0.288713
[11]	valid_0's l1: 0.394198	valid_0's l2: 0.254852
[12]	valid_0's l1: 0.369077	valid_0's l2: 0.226904
[13]	valid_0's l1: 0.346546	valid_0's l2: 0.203874
[14]	valid_0's l1: 0.326653	valid_0's l2: 0.18492
[15]	valid_0's l1: 0.308829	valid_0's l2: 0.16907
[16]	valid_0's l1: 0.293734	valid_0's l2: 0.156022
[17]	valid_0's l1: 0.279842	valid_0's l2: 0.144786
[18]	valid_0's l1: 0.267067	valid_0's l2: 0.135346
[19]	valid_0's l1: 0.256647	valid_0's l2: 0.127578
[20]	valid_0's l1: 0.246822	valid_0's l2: 0.12

LGBMRegressor(bagging_fraction=0.8, boosting_type='gbdt', class_weight=None,
              colsample_bytree=1.0, importance_type='split', learning_rate=0.1,
              max_depth=-1, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=20,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

In [99]:
mean_absolute_error(getNat(gbm.predict(x_test)),getNat(y_test))

759.4037068107085